In [ ]:
import sys
import requests
import threading

import boto3
from boto3.s3.transfer import TransferConfig
from tqdm.auto import tqdm

# Defines

In [ ]:
bucket = ""  # Output bucket

In [ ]:
s3 = boto3.client("s3")

In [ ]:
class ProgressPercentage(object):

    def __init__(self, file):
        self._filename = "filename"
        self._size = int(file.headers.get('Content-Length', 0))
        self._seen_so_far = 0
        self._lock = threading.Lock()

    def __call__(self, bytes_amount):
        # To simplify, assume this is hooked up to a single filename
        with self._lock:
            self._seen_so_far += bytes_amount
            percentage = (self._seen_so_far / self._size) * 100
            sys.stdout.write(
                "\r%s  %s / %s  (%.2f%%)" % (
                    self._filename, self._seen_so_far, self._size,
                    percentage))
            sys.stdout.flush()

# `match_complete` file

In [ ]:
url = "https://ftp.ebi.ac.uk/pub/databases/interpro/current_release/match_complete.xml.gz"
key = "interpro/raw/match_complete.xml.gz"

In [ ]:
with requests.get(url, stream=True) as r:
    r.raise_for_status()

    file_size = int(r.headers.get('Content-Length', 0))
    part_size = file_size // 5000

    s3.upload_fileobj(
        r.raw, bucket, key,
        Callback=ProgressPercentage(r),
        Config=TransferConfig(multipart_chunksize=part_size)
    )

# `protein2ipr.dat.gz` file
All UniProtKB proteins and the InterPro entries and individual signatures they match, in a tab-delimited format.

In [ ]:
url = "https://ftp.ebi.ac.uk/pub/databases/interpro/current_release/protein2ipr.dat.gz"
key = "interpro/raw/protein2ipr.dat.gz"

In [ ]:
with requests.get(url, stream=True) as r:
    r.raise_for_status()

    file_size = int(r.headers.get('Content-Length', 0))
    part_size = file_size // 5000

    s3.upload_fileobj(
        r.raw, bucket, key,
        Callback=ProgressPercentage(r),
        Config=TransferConfig(multipart_chunksize=part_size)
    )